# Index

- **Python Libraries**
- **Task 3**
    - *Reading Preprocessed Datasets*
    - *Exercise 1*



# Pyhton Libraries

In [1]:
import torch
import numpy as np
import datetime as dt
import random

In [2]:
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

Tensorflow/Keras: 2.12.0
pandas: 1.5.3
numpy: 1.22.4
sklearn: 1.2.2
plotly: 5.13.1


In [3]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Task 2

## Reading Preprocessed Datasets

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
read_data_path  = "../content/drive/MyDrive/Data3"

ap   = pd.read_csv(read_data_path+'/anagraficapazientiattivi.csv', header=0 ,names=['idcentro','idana', 'idcopy','sesso','annodiagnosidiabete','tipodiabete','annonascita','annoprimoaccesso','annodecesso','label'])
diag = pd.read_csv(read_data_path+'/diagnosi.csv', header=0 ,names=['idcentro','idana','idcopy','data','codiceamd','valore'])
elp  = pd.read_csv(read_data_path+'/esamilaboratorioparametri.csv', header=0 ,names=['idcentro','idana', 'idcopy','data','codiceamd','valore'])
ei   = pd.read_csv(read_data_path+'/esamistrumentali.csv', header=0 ,names=['idcentro','idana','idcopy','data','codiceamd','valore'])
pdf  = pd.read_csv(read_data_path+'/prescrizionidiabetefarmaci.csv', header=0 ,names=['idcentro','idana','idcopy','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf = pd.read_csv(read_data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','idcopy','data','codiceamd','valore'])
pnd  = pd.read_csv(read_data_path+'/prescrizioninondiabete.csv', header=0 ,names=['idcentro','idana','idcopy','data','codiceamd','valore'])
ap.head()

,idcentro,idana,idcopy,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label
0,1,60,1,F,1987.0,5,1923,1987.0,2010.0,0
1,1,74,1,M,1995.0,5,1930,1996.0,2013.0,0
2,1,143,1,M,1991.0,5,1934,1991.0,2015.0,0
3,1,195,1,F,1976.0,5,1936,1997.0,2015.0,0
4,1,230,1,F,1990.0,5,1923,1993.0,2018.0,0


## Exercise 1

Some macro-events $e_h^i$
are represented as visits within a patient trajectory $p_i$ ∈ P. These visits
might contain other micro-events $e_h^i$ = {$e^i_{h,1}$
, . . . , $e^i_{h,l}$} including lab examinations, autonomous
glucose tests, and so on. While the order of the macro-events $e^i_h$ ∈ $p_i$
is important - i.e., the
timestamp h indexes the events in time - the order of the micro-events $e^i_{h,j}$ ∈ $e^i_h$
∀j ∈ [1, l] is
not essential in being represented in the learnt latent space of your prediction model. Devise a
prediction strategy that does not consider the order of the micro-events to perform predictions.

Tenemos una secuencia de eventos, deep learning que hemos visto para sequence models son las RNN (¿SERIA MANY-TO-ONE porque tenemos muchos dato y una unica salida?), GRU y el LSTM

In [6]:
# At first, we merge with the patient data
diag1 = pd.merge(diag, ap, on=['idcentro','idana','idcopy'])
elp1 = pd.merge(elp,ap,on = ['idcentro','idana','idcopy'] )
ei1= pd.merge(ei,ap,on = ['idcentro','idana','idcopy'] )
pdf1 = pd.merge(pdf,ap,on = ['idcentro','idana','idcopy'] )
pdnf1 = pd.merge(pdnf,ap,on = ['idcentro','idana','idcopy'] )
pnd1 = pd.merge(pnd,ap,on = ['idcentro','idana','idcopy'] )

features_gru=pd.concat([diag1,elp1],axis=0)
features_gru=pd.concat([features_gru,ei1],axis=0)
features_gru=pd.concat([features_gru,pdf1],axis=0)
features_gru=pd.concat([features_gru,pdnf1],axis=0)
features_gru=pd.concat([features_gru,pnd1],axis=0)

# We create a single ID column that combines the other three:
features_gru['id'] = features_gru.apply(lambda x: f"{x['idcentro']}_{x['idana']}_{x['idcopy']}",axis=1)

# We reorder the columns
features_gru = features_gru[['id']+[x for x in features_gru.columns if x!='id']]

# We drop the other ID_columns
features_gru.drop(columns=['idcentro','idana','idcopy'], inplace=True)

features_gru.head()


,id,data,codiceamd,valore,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label,codiceatc,quantita,idpasto,descrizionefarmaco
0,1_60_1,2009-07-16,AMD302,S,F,1987.0,5,1923,1987.0,2010.0,0,NaN,NaN,NaN,NaN
1,1_60_1,2009-04-29,AMD047,410.0,F,1987.0,5,1923,1987.0,2010.0,0,NaN,NaN,NaN,NaN
2,1_60_1,2009-04-29,AMD247,412,F,1987.0,5,1923,1987.0,2010.0,0,NaN,NaN,NaN,NaN
3,1_60_1,2008-12-03,AMD130,S,F,1987.0,5,1923,1987.0,2010.0,0,NaN,NaN,NaN,NaN
4,1_60_1,2008-12-03,AMD247,366.41,F,1987.0,5,1923,1987.0,2010.0,0,NaN,NaN,NaN,NaN


In [7]:
# We categorize the columns that contain text
categorical_columns = ['codiceamd', 'valore', 'sesso','codiceatc','descrizionefarmaco']
for col in categorical_columns:
    features_gru[col] = features_gru[col].astype('category')
    features_gru[col] = features_gru[col].cat.codes

# We convert every columns into float type
numerical_columns = [col for col in features_gru.columns if col not in ['id','data']]
for col in numerical_columns:
    features_gru[col] = features_gru[col].astype('float')

features_gru.head(10)

,id,data,codiceamd,valore,sesso,annodiagnosidiabete,tipodiabete,annonascita,annoprimoaccesso,annodecesso,label,codiceatc,quantita,idpasto,descrizionefarmaco
0,1_60_1,2009-07-16,101.0,5710.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0
1,1_60_1,2009-04-29,35.0,5250.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0
2,1_60_1,2009-04-29,97.0,5263.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0
3,1_60_1,2008-12-03,79.0,5710.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0
4,1_60_1,2008-12-03,97.0,5163.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0
5,1_60_1,2008-12-03,97.0,5160.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0
6,1_60_1,2008-11-10,101.0,5710.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0
7,1_60_1,2007-07-29,101.0,5710.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0
8,1_60_1,2007-02-02,36.0,5136.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0
9,1_60_1,2007-02-02,97.0,5263.0,0.0,1987.0,5.0,1923.0,1987.0,2010.0,0.0,-1.0,NaN,NaN,-1.0


In [8]:
# We select the column label to obtain the Y_train
# We drop the columns id, data and label to obtain the X_train
Y_t= features_gru['label']
features_gru.drop(columns=['id','data','label'], inplace=True)
X_train=features_gru

In [9]:
# We transform the train dataframes to train tensors
X_train=torch.tensor(X_train.values)
Y_train=torch.tensor(Y_t.values)

In [10]:
X_train.shape

torch.Size([889317, 12])

In [11]:
# We transform the train tensors to train tensorflows
X_train=tf.cast(X_train,tf.float32)
Y_train=tf.cast(Y_train,tf.float32)

In [12]:
# The GRU architecture
regressorGRU = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.4))
# Second GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.4))
# Third GRU layer
regressorGRU.add(GRU(units=50, activation='tanh'))
regressorGRU.add(Dropout(0.2))
# The output layer
regressorGRU.add(Dense(units=1))
# Compiling the RNN
regressorGRU.compile(optimizer=SGD(learning_rate=0.15, decay=1e-7, momentum=0.6, nesterov=False),loss='mean_squared_error')
# Fitting to the training set
regressorGRU.fit(X_train,Y_train,epochs=25,batch_size=150)

Epoch 1/25
2551/5929 [===========>..................] - ETA: 2:37 - loss: nan

KeyboardInterrupt: ignored